# Сегментация
---

- Отличие сегментации от детекции в том, что мы для каждого пикселя определяем принадлежит ли он нашемму объекту или нет.

**Semantic segmentation** - определить все объекты,  которые принадлежат одному классу, например просто выделить всех людей и закрасить их одним цветом 

**Instance segmentation** - определить все объекты, которые принадлежат одному классу, а затем еще среди них определить разные "экземплляры". То есть найти всех людей на картинке и разных людей в разный цвет раскрасить

Можно сегментировать вообще все, а можно только некоторые объекты

**Задача сегментации:** по входной картинке, для каждого пикселя картинки определить к какому классу он принадлежит, то есть мы должны заранее знать на какие классы мы хотим сегментировать картинку.

- Сетка будет выдавать для каждого пикселя вероятность принадлежности к каждому классу, то есть выданная картинка будет как бы: HxWxClasses, то есть для каждого пикселя будет вектор вероятностей/логитов для классов.

- Возвращать надо картинку такого же размера

Ну и просто к каждому пикселю будем применять какой-то классификационный лосс

## Идеи решения.

- Точно так же как мы при помощи лосса застявляем сетку обучаться на классификацию, мы можем заставить ее обучаться на сегментацию объектов

#### Sliding window

вокруг каждого пикселя берем окно вокруг него и затем для такого окна решаем простую задачу классификации изображения. Пикселю тогда просто присваиваем вектор вероятностей принадлежности, который получился для такого окна. Так мы все пиксели классифицируем

**Недостатки:**
- вычислительно очень дорого
- отдельные окна не шейрят информацию между собой о частях картинки

#### Fully-conv network (без FC слоев)

<img src="https://drive.google.com/uc?export=view&id=1XZl0m0UssG4lMUv2M4R_SP6LTJu4bZ98" width="1000">

Каждая фича мапа перед выходом - это слой из вероятности принадлежности каждого пикселя к классу, которому эта фича мапа соответствует. Мы учим сетку используя лосс, чтобы полученные фича мапы были картами сегментации

Короче сетка при помощи лосс функции сможет научиться сегментировать, она поймет, что лосс растет, если мы на выходе для 1 фича мапы выделяем только 1 класс, для 2 фича мапы выделяем только 2 класс и тд.

- вычислительно еще дороже даже чем предыдущий способ

Надо как-то сжимать информацию

##### CNN + upsampling - тоже подход FCN

Учим сетку так, чтобы выходные фича мапы были картами сегментации для какого-то класса

испольховать просто архитектуру сверточной сети, там параметров надо гораздо меньше - размер сильно уменьшается от слоя к слою, а затем мы просто выходные фича увеличим в размерах - сделаем upsampling и потом объединим их в одну сегментированную картинку

**Проблемы:** (из-за разрушения пространственной информации)

- upsampling плохо восстанавливает информацию

- В данном подходе мы из-за upsampling, пулингов, большого страйда при свертке теряем пространственную информацию, это было полезно для простой задачи классификации, потому что нам надо было детектить собаку вне зависимости от того в какой части картинки она находится, но для сегментации это плохо, потому что нам надо сегментировать именно исходную картинку

- Scale variability - котов разных размеров мы не сможем успешно сегментировать

Сегментация сверточной сетью очень шумная и результат довольно плохой - процентов 70 пикселей правильно сегментированы, а спорные зоны вообще все неправильно сегментируются. Но! Люди придумали postprocessing (CRF)

#### Решение проблемы плохого восстановления информации при upsampling:

##### DeConvNet

<img src="https://drive.google.com/uc?export=view&id=15fFR8taeQNH3psMs0MA3LbyqA7InyMLd" width="1000">

никакой обратной свертки там не происходит, там происходит **Transposed convolution = upsampling + convolution**:

- Маленькую фича мапу растягиваем upsampling-ом со страйдом - потом применяем ядро свертки к тому что заапсемплили, таким образом мы увеличиваем размер, но также еще какие-то параметры для ядер "deconvolution" обучаем при увеличивании размеров, однако по количеству параметров это на несколько порядков быстрее учится, чем если просто все гнать с огромным разрешением на всех свертках.

https://youtu.be/K73tZxH9nvE?t=533 - анимация того как происходит

идея в том, чтобы не просто upsampling сделать, потеряв всю информацию, а попытаться как-то при увеличении разрешения картинки получить еще информацию

#### Варианты upsampling:

1) Nearest neihbours

2) Bed of Nails - втыкаем шапки гвоздей из нулей в фича мапу

3) Bilinear

4) Max-unpooling - происходит симметрично пуллингу, который происходил на прямом пути

Сетке не обязательно быть симметричной, также можно применть batch-norm и функции активации так же как при обычной свертке.

Для меня удобно мыслить это как две отдельные части сетки:
- сверточная часть сети
- "часть сети, которая увеличивает размер изображения" - upsampling + convolution = deconvolution (опять же, без такого мы либо просто теряем кучу информации, если разрешение фича мапов сильно уменьшается, либо вычисления будут просто неподъемными если все сверточные слои будут над картинками полного разрешения)

#### Решение проблемы разрушения пространственной информации:

- Dialated convolutions - мы upsampl'им как бы фильтр:

$$
y[i] = \sum_{k=1}^K x[i + r \cdot k]w[k]
$$

r = 1 соответствует обычной свертке, если больше, то это как бы свертка с дырками - через (r-1) суммируем при свертке

K - это размер фильтра

$y[i]$ - элемент карты активации после свертки.


$x[i]$ - элемент 

$w[i]$ - вес в фильтре на этой позиции

Моя немного улучшенная запись запись для свертки:


$$
y[i][j] = \sum_{m=1}^N\sum_{k=1}^N x[i + r \cdot k][j + r \cdot m]w[k][m]
$$

$y[i][j]$ - элемент карты активации после свертки.

$x[\alpha][\beta]$ - элемент исходной фича мапы, которую сворачиваем

$w[\gamma][\delta]$ - вес в фильтре на этой позиции

сворачиваем ядром NxN, r=1 - это обычная свертка, если r повышаем, это Dialated convolution

При проходе ядром: индекс k - быстрый индекс - номер столбца, m - медленный индекс, номер строки

- В торче у обычной свертки есть параметр 'dialation=1' - он по умолчанию стоит 1 - то есть это простая свертка происходит

При помощи такой свертки - чем больше r, тем больше будет область картинки, которую ядро накрывает

**Что в итоге имеем:**

- Идея которую реализует такой подход заключается в том, чтобы иметь такое же количество параметров у ядра, однако каждая свертка будет содержать больше информации о пространственном положении, потому что мы как бы большую область накрываем, но с дырками. А так же наша выходная фича мапа будет маленького размера, мы как будто уже учли при такой свертке пуллинг и страйд при обычной свертке

- Также это заменяет нам использование stride при свертке, а также pooling слоя, потому что все это уже реализует такой метод свертки, то есть нам не приходится использовать большой страйд и макс пуллинг

- Использовать это естественно надо только, когда нам надо сохранять пространственную информацию об объектах, поэтому при простой классификации изображений такое использовать наоборот не нужно, там нам наоборот желательно как можно меньше простравнственных связей сохранять, чтобы не переобучаться и уметь в любой части экрана кота распознать

#### Решение проблемы Scale variabilty:

Dialated convolutions разных размеров могут выделять информацию об объектах разных размеров

Здесь можно использовать идею Inception - использовать параллельно ветки с разными dialation_rate:

<img src="https://drive.google.com/uc?export=view&id=1YZXkCqnNliAhwNTgfDPIPj8GREV6ZLnu" width="800">

Можно параллельно использовать разные пуллинги, потом апсемплить и конкатенировать:

<img src="https://drive.google.com/uc?export=view&id=1LhhWkj_qt7hm6Y2cPuneJncdUU4DgXxo" width="800">

Существует метод постпроцессинга Conditional Random Field - CRF, он позволяет еще более детальными сделать границы классов

#### Архитектуры для сегментации:

https://youtu.be/K73tZxH9nvE?t=1860

## Архитектура UNet

<img src="https://drive.google.com/uc?export=view&id=1ZcrqFadpa7RGdR8MxY7sTNXyHnLgrGXv" width="1000">

Здесь реализован skip-connection тем же методом, как у DenseNet

Сетка симметричная, конкатенация происходит вместе с симметричными up-conv слоями, слоев на выходе после conv - так у нас сразу фича мапы с одинаковым разрешением конкатенируются

Левая часть - Encoder - extracts what information

Правая часть - Decoder - recovers where information

**Что дают такие skip connection'ы:**

- Они позволяют нам получать некоторый доступ к пространственной информации при декодировании

**Проблема сегментации пикселей на краях изображений**

Появляется проблема того, что очень мало информации о краях изображения из-зак того как работает свертка

Overlap-tile strategy: https://youtu.be/yEuIV5FsRMs?t=635

- Чтобы ее решить мы просто зеркалим несколько пикселей по краям - как бы зеркальный паддинг, и тогда у нас все прекрасно сработает - в центре будет как раз вся картинка до паддинга и вся информация о краях при свертках будет получена
  
- И если зеркалить, то ничего страшного как бы при проходе ядром по этим отзеркаленным местам не произойдет

**Необходимо выбрать другой лосс**

При обучении у нас очень много места занимает фон или внутренности объектов, но нам-то важнее всего классификация тоненькой границы между объектами - это самое важное, поэтому надо как-то лосс корректировать

- Решение - взвешенный лосс - будем скейлить лосс в зависимости от нахождения на границе:



<img src="https://drive.google.com/uc?export=view&id=1apMoQABh4IPl2kCfcBy1GebwhyTU4ohs" width="1200">

##### Пара слов о Mask R-CNN

Сначала решаем задачу детекции - она очевидно, вычислительно гораздо проще задачи сегментации. А затем мы внутри bounding boxes выполняем задачу сегментации